In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st
import spotipy 

# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Importar funciones de gestión
from utils.chat_gpt_manager import (
    get_chords_chat,
    create_song
)
from utils.spotify_manager import (
    get_similar_songs,
    get_audio_analysis,
    get_audio_features,
    get_song_and_artist
)
from utils.functions import (
    add_info_list_df,
    convert_chords_with_minor,
    standardize_chord_symbols,
    transpose_chords_to_key,
    check_track_csv,
    filter_existing_vector_id,
    to_embbed_text_column,
    expand_and_remove_original_columns,
    drop_columns_with_many_nulls,
    get_user_input,
    handle_input_search,
    calc_diff,
    export_unique_rows,
    create_metadata_column,
    clean_metadata_column,
    insert_into_pinecone,
    create_dataframe_for_pinecone,
    realizar_consulta

    
)

from utils.pinecone_manager import (
    initialize_pinecone, 
    setup_index
)

from utils.lyrics_manager import insert_lyrics_db

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Initizalize pinecone

In [2]:
pc = initialize_pinecone()

In [3]:
index = setup_index(pc)

Index 'song-starter-index' already exists, skipping creation.


In [4]:
index_name= pc.list_indexes().indexes[0].name

# API de Spotify

Aquí se utiliza la API de Spotify para obtener recomendaciones de canciones basadas en una canción dada. 

Input:

In [5]:
input_song, input_artist = get_user_input()

In [6]:
# Llamar a la función para obtener el track ID
input_track_id=handle_input_search(input_song, input_artist)

In [7]:
input_song , input_artist = get_song_and_artist(input_track_id)

In [8]:
input_data = pd.DataFrame({'id': [input_track_id]})
input_data['name'] = input_song
input_data['artists'] = input_artist

In [9]:
input_data

,id,name,artists
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles


Similar songs

In [10]:
tracks_data = get_similar_songs(input_track_id, 5)

In [11]:
for track in tracks_data:
    print(track['name'])


Mr. Tambourine Man
I Saw Her Standing There - Remastered 2009
Crocodile Rock
Long As I Can See The Light
Maria


In [12]:
tracks_data = pd.DataFrame(tracks_data)

In [13]:
tracks_data.head(2)

,id,name,artists
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles


MIRAR ESTO, cómo evitar que a partir de aquí todo de error si ya están todas en pinecone

In [14]:
filter_existing_vector_id(index, tracks_data)            

In [15]:

tracks_data.reset_index(drop=True, inplace=True)

In [16]:
tracks_data

,id,name,artists
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie


In [17]:
csv= pd.read_csv('mi_archivo_temporal.csv')

In [18]:
if len(tracks_data)>0:
    df=check_track_csv(tracks_data, csv)

In [19]:
if len(df)>0:    
    df.reset_index(drop=True, inplace=True)

# API de ChatGPT

Aquí se utiliza la API de Chat GPT, para obtener los acordes e instrumentos de las canciones

Input:

In [20]:
add_info_list_df(input_data, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]"


In [21]:
add_info_list_df(input_data, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]"


Related tracks:

In [22]:
add_info_list_df(df, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, G, A, D]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[A, D, A, E, A, D, A, E]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, B, E, A, B, E, A, A]"
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,"[A, A, D, E, A, A, D, E]"
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,"[C, G, Am, F, C, G, F, C]"


In [23]:
add_info_list_df(df, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, G, A, D]","[D, A, G, D, D, A, G, D]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[A, D, A, E, A, D, A, E]","[E, A, B, E, C#m, F#m, B, E]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, B, E, A, B, E, A, A]","[A, D, A, E, A, D, A, E]"
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,"[A, A, D, E, A, A, D, E]","[A, E, F#m, D, A, E, D, A]"
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,"[C, G, Am, F, C, G, F, C]","[A, D, E, A, G, C, D, G]"


# API de Spotify para nuevas columnas

Input:

In [24]:
get_audio_features(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0


In [25]:
get_audio_analysis(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339


In [26]:
insert_lyrics_db(input_data)

Timeout o fallo! Usando GPT para obtener el tema para The Beatles - Let It Be - Remastered 2009


(                       id                         name      artists  \
 0  7iN1s7xHE4ifF5povM6A48  Let It Be - Remastered 2009  The Beatles   
 
                       chords                instruments  speechiness  \
 0  [C, G, Am, F, C, G, F, C]  [C, G, Am, F, C, G, F, C]       0.0322   
 
    instrumentalness  key  mode    tempo  loudness  \
 0                 0    0     1  143.462    -8.339   
 
                                                theme  
 0  The song "Let It Be" by The Beatles conveys th...  ,
 'The song "Let It Be" by The Beatles conveys themes of comfort and acceptance in the face of uncertainty and challenges. It emphasizes the importance of letting go and finding peace amidst life\'s difficulties, often invoking the idea of guidance and reassurance in times of trouble. The refrain serves as a reminder that, even in moments of despair, there is a sense of hope and solace that can be found by simply allowing things to unfold naturally. Overall, the song promotes a m

In [27]:
input_data['chords'].dropna(inplace=True)
input_data = input_data[input_data['chords'].apply(lambda x: x != [])]

In [28]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th..."


Related tracks:

In [29]:
get_audio_features(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, G, A, D]","[D, A, G, D, D, A, G, D]",0.0676,0.000000
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[A, D, A, E, A, D, A, E]","[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, B, E, A, B, E, A, A]","[A, D, A, E, A, D, A, E]",0.0344,0.000015
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,"[A, A, D, E, A, A, D, E]","[A, E, F#m, D, A, E, D, A]",0.0484,0.000153
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,"[C, G, Am, F, C, G, F, C]","[A, D, E, A, G, C, D, G]",0.0597,0.002430


In [30]:
get_audio_analysis(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, G, A, D]","[D, A, G, D, D, A, G, D]",0.0676,0.000000,5,1,177.515,-14.545
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[A, D, A, E, A, D, A, E]","[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000,4,1,160.109,-9.835
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, B, E, A, B, E, A, A]","[A, D, A, E, A, D, A, E]",0.0344,0.000015,4,0,149.908,-7.284
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,"[A, A, D, E, A, A, D, E]","[A, E, F#m, D, A, E, D, A]",0.0484,0.000153,11,1,63.299,-10.537
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,"[C, G, Am, F, C, G, F, C]","[A, D, E, A, G, C, D, G]",0.0597,0.002430,9,1,159.656,-6.926


In [31]:
insert_lyrics_db(df)

Timeout o fallo! Usando GPT para obtener el tema para The Beatles - I Saw Her Standing There - Remastered 2009


(                       id                                        name  \
 0  3RkQ3UwOyPqpIiIvGVewuU                          Mr. Tambourine Man   
 1  3KiexfmhxHvG5IgAElmTkd  I Saw Her Standing There - Remastered 2009   
 2  6WCeFNVAXUtNczb7lqLiZU                              Crocodile Rock   
 3  1Vu8p3k4rfbETQCjTGjmzg                 Long As I Can See The Light   
 4  7bOYOD0d7ByMTXvJnQVl8d                                       Maria   
 
                         artists                     chords  \
 0                     Bob Dylan   [D, A, G, D, D, G, A, D]   
 1                   The Beatles   [A, D, A, E, A, D, A, E]   
 2                    Elton John   [A, B, E, A, B, E, A, A]   
 3  Creedence Clearwater Revival   [A, A, D, E, A, A, D, E]   
 4                       Blondie  [C, G, Am, F, C, G, F, C]   
 
                     instruments  speechiness  instrumentalness  key  mode  \
 0      [D, A, G, D, D, A, G, D]       0.0676          0.000000    5     1   
 1  [E, A, B, E, C

In [32]:
df_vectorial= df.copy()
df_vectorial['chords'].dropna(inplace=True)
df_vectorial = df_vectorial[df_vectorial['chords'].apply(lambda x: x != [])]

# Separate columns

Input:

In [33]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th..."


In [34]:
input_data['chords_1'] = input_data['chords'].apply(lambda x: x[:4]) 
input_data['chords_2'] = input_data['chords'].apply(lambda x: x[4:])

In [35]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]"


In [36]:
input_data['chords_1'] = input_data['chords_1'].apply(standardize_chord_symbols)
input_data['chords_2'] = input_data['chords_2'].apply(standardize_chord_symbols)

In [37]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]"


In [38]:
input_data['chords_1_numeric'] = input_data['chords_1'].apply(convert_chords_with_minor)
input_data['chords_2_numeric'] = input_data['chords_2'].apply(convert_chords_with_minor)

In [39]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]"


In [40]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    input_data[f'normal_{col}'] = input_data.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[0, 7, 21, 5]","[0, 7, 5, 0]"


In [41]:
for col in col_chords:
    input_data[f'diff_{col}'] = input_data.apply(lambda row: calc_diff(row[col]), axis=1)

input_data


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[0, 7, 21, 5]","[0, 7, 5, 0]","[7, 14, -16]","[7, -2, -5]"


In [42]:
input_data.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [43]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[0, 7, 21, 5]","[0, 7, 5, 0]","[7, 14, -16]","[7, -2, -5]"


Related tracks:

In [44]:
df_vectorial['chords_1'] = df_vectorial['chords'].apply(lambda x: x[:4]) 
df_vectorial['chords_2'] = df_vectorial['chords'].apply(lambda x: x[4:])
df_vectorial.drop(columns=['chords'], inplace=True)

In [45]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, A, G, D]",0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me,\n...","[D, A, G, D]","[D, G, A, D]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...","[A, D, A, E]","[A, D, A, E]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, D, A, E, A, D, A, E]",0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young\nMe and Suzie h...,"[A, B, E, A]","[B, E, A, A]"


In [46]:
# Nos ocupamos de bemoles o sostenidos para estandarizarlos
df_vectorial['chords_1'] = df_vectorial['chords_1'].apply(standardize_chord_symbols)
df_vectorial['chords_2'] = df_vectorial['chords_2'].apply(standardize_chord_symbols)

In [47]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, A, G, D]",0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me,\n...","[D, A, G, D]","[D, G, A, D]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...","[A, D, A, E]","[A, D, A, E]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, D, A, E, A, D, A, E]",0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young\nMe and Suzie h...,"[A, B, E, A]","[B, E, A, A]"


In [48]:
# Convertimos los acordes a valores numéricos
df_vectorial['chords_1_numeric'] = df_vectorial['chords_1'].apply(convert_chords_with_minor)
df_vectorial['chords_2_numeric'] = df_vectorial['chords_2'].apply(convert_chords_with_minor)

In [49]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, A, G, D]",0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me,\n...","[D, A, G, D]","[D, G, A, D]","[2, 9, 7, 2]","[2, 7, 9, 2]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...","[A, D, A, E]","[A, D, A, E]","[9, 2, 9, 4]","[9, 2, 9, 4]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, D, A, E, A, D, A, E]",0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young\nMe and Suzie h...,"[A, B, E, A]","[B, E, A, A]","[9, 11, 4, 9]","[11, 4, 9, 9]"


In [50]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    df_vectorial[f'normal_{col}'] = df_vectorial.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, A, G, D]",0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me,\n...","[D, A, G, D]","[D, G, A, D]","[2, 9, 7, 2]","[2, 7, 9, 2]","[-3, 4, 2, -3]","[-3, 2, 4, -3]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...","[A, D, A, E]","[A, D, A, E]","[9, 2, 9, 4]","[9, 2, 9, 4]","[5, -2, 5, 0]","[5, -2, 5, 0]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, D, A, E, A, D, A, E]",0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young\nMe and Suzie h...,"[A, B, E, A]","[B, E, A, A]","[9, 11, 4, 9]","[11, 4, 9, 9]","[5, 7, 0, 5]","[7, 0, 5, 5]"
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,"[A, E, F#m, D, A, E, D, A]",0.0484,0.000153,11,1,63.299,-10.537,"Put a candle in the window, \n'cause I feel I'...","[A, A, D, E]","[A, A, D, E]","[9, 9, 2, 4]","[9, 9, 2, 4]","[-2, -2, -9, -7]","[-2, -2, -9, -7]"
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,"[A, D, E, A, G, C, D, G]",0.0597,0.002430,9,1,159.656,-6.926,"She moves like she don't care\nSmooth as silk,...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-9, -2, 12, -4]","[-9, -2, -4, -9]"


In [51]:
for col in col_chords:
    df_vectorial[f'diff_{col}'] = df_vectorial.apply(lambda row: calc_diff(row[col]), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, A, G, D]",0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me,\n...","[D, A, G, D]","[D, G, A, D]","[2, 9, 7, 2]","[2, 7, 9, 2]","[-3, 4, 2, -3]","[-3, 2, 4, -3]","[7, -2, -5]","[5, 2, -7]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...","[A, D, A, E]","[A, D, A, E]","[9, 2, 9, 4]","[9, 2, 9, 4]","[5, -2, 5, 0]","[5, -2, 5, 0]","[-7, 7, -5]","[-7, 7, -5]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, D, A, E, A, D, A, E]",0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young\nMe and Suzie h...,"[A, B, E, A]","[B, E, A, A]","[9, 11, 4, 9]","[11, 4, 9, 9]","[5, 7, 0, 5]","[7, 0, 5, 5]","[2, -7, 5]","[-7, 5, 0]"
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,"[A, E, F#m, D, A, E, D, A]",0.0484,0.000153,11,1,63.299,-10.537,"Put a candle in the window, \n'cause I feel I'...","[A, A, D, E]","[A, A, D, E]","[9, 9, 2, 4]","[9, 9, 2, 4]","[-2, -2, -9, -7]","[-2, -2, -9, -7]","[0, -7, 2]","[0, -7, 2]"
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,"[A, D, E, A, G, C, D, G]",0.0597,0.002430,9,1,159.656,-6.926,"She moves like she don't care\nSmooth as silk,...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-9, -2, 12, -4]","[-9, -2, -4, -9]","[7, 14, -16]","[7, -2, -5]"


In [52]:
df_vectorial.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [53]:
df_vectorial = df_vectorial.fillna(0)

In [54]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,"[D, A, G, D, D, A, G, D]",0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me, I...","[D, A, G, D]","[D, G, A, D]","[2, 9, 7, 2]","[2, 7, 9, 2]","[-3, 4, 2, -3]","[-3, 2, 4, -3]","[7, -2, -5]","[5, 2, -7]"
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,"[E, A, B, E, C#m, F#m, B, E]",0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...","[A, D, A, E]","[A, D, A, E]","[9, 2, 9, 4]","[9, 2, 9, 4]","[5, -2, 5, 0]","[5, -2, 5, 0]","[-7, 7, -5]","[-7, 7, -5]"
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,"[A, D, A, E, A, D, A, E]",0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young Me and Suzie ha...,"[A, B, E, A]","[B, E, A, A]","[9, 11, 4, 9]","[11, 4, 9, 9]","[5, 7, 0, 5]","[7, 0, 5, 5]","[2, -7, 5]","[-7, 5, 0]"


In [55]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[0, 7, 21, 5]","[0, 7, 5, 0]","[7, 14, -16]","[7, -2, -5]"


# Backup csv

In [56]:
csv_file = 'mi_archivo_temporal.csv'

# Llamar la función para exportar
export_unique_rows(df_vectorial, csv_file)



/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:277: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: str(x) if isinstance(x, list) else x)
/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:283: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)


# Expand columns

In [57]:
# Especificar las columnas que contienen listas (excepto 'embedding')
columns_to_expand = ['instruments', 'chords_1', 'chords_2', 
                      'chords_1_numeric', 'chords_2_numeric', 
                      'normal_chords_1_numeric', 'normal_chords_2_numeric', 
                      'diff_chords_1_numeric', 'diff_chords_2_numeric']

Input:

In [58]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,"The song ""Let It Be"" by The Beatles conveys th...","[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[0, 7, 21, 5]","[0, 7, 5, 0]","[7, 14, -16]","[7, -2, -5]"


In [59]:
columns_to_expand

['instruments',
 'chords_1',
 'chords_2',
 'chords_1_numeric',
 'chords_2_numeric',
 'normal_chords_1_numeric',
 'normal_chords_2_numeric',
 'diff_chords_1_numeric',
 'diff_chords_2_numeric']

In [60]:
input_expanded = expand_and_remove_original_columns(input_data, columns_to_expand)

In [61]:
input_expanded

,id,name,artists,chords,speechiness,instrumentalness,key,mode,tempo,loudness,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,"[C, G, Am, F, C, G, F, C]",0.0322,0,0,1,143.462,-8.339,...,0,7,5,0,7,14,-16,7,-2,-5


In [62]:
#elimino las columnas que tengan solo valores nulos
input_expanded = input_expanded.dropna(axis=1, how='all')

Related tracks:

In [63]:
df_vectorial_expanded = expand_and_remove_original_columns(df_vectorial, columns_to_expand)

In [64]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me, I...",...,-3,2,4,-3,7,-2,-5,5,2,-7
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...",...,5,-2,5,0,-7,7,-5,-7,7,-5
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young Me and Suzie ha...,...,7,0,5,5,2,-7,5,-7,5,0
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,0.0484,0.000153,11,1,63.299,-10.537,"Put a candle in the window, 'cause I feel I'v...",...,-2,-2,-9,-7,0,-7,2,0,-7,2
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,0.0597,0.002430,9,1,159.656,-6.926,"She moves like she don't care Smooth as silk, ...",...,-9,-2,-4,-9,7,14,-16,7,-2,-5


In [65]:
#elimino las columnas que tengan solo valores nulos
df_vectorial_expanded = df_vectorial_expanded.dropna(axis=1, how='all')

In [66]:
drop_columns_with_many_nulls(df_vectorial_expanded, 15)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me, I...",...,-3,2,4,-3,7,-2,-5,5,2,-7
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...",...,5,-2,5,0,-7,7,-5,-7,7,-5
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young Me and Suzie ha...,...,7,0,5,5,2,-7,5,-7,5,0
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,0.0484,0.000153,11,1,63.299,-10.537,"Put a candle in the window, 'cause I feel I'v...",...,-2,-2,-9,-7,0,-7,2,0,-7,2
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,0.0597,0.002430,9,1,159.656,-6.926,"She moves like she don't care Smooth as silk, ...",...,-9,-2,-4,-9,7,14,-16,7,-2,-5


In [67]:
df_vectorial_expanded.head(3)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me, I...",...,-3,2,4,-3,7,-2,-5,5,2,-7
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...",...,5,-2,5,0,-7,7,-5,-7,7,-5
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young Me and Suzie ha...,...,7,0,5,5,2,-7,5,-7,5,0


# Create column for the text to create the embeddings

Input:

In [68]:
input_expanded=to_embbed_text_column(input_expanded)

In [69]:
input_expanded['to_embbed_text']

0    name: Let It Be - Remastered 2009 artists: The...
Name: to_embbed_text, dtype: object

Related songs:

In [70]:
df_vectorial_expanded=to_embbed_text_column(df_vectorial_expanded)

In [71]:
df_vectorial_expanded


,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me, I...",...,2,4,-3,7,-2,-5,5,2,-7,name: Mr. Tambourine Man artists: Bob Dylan sp...
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...",...,-2,5,0,-7,7,-5,-7,7,-5,name: I Saw Her Standing There - Remastered 20...
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young Me and Suzie ha...,...,0,5,5,2,-7,5,-7,5,0,name: Crocodile Rock artists: Elton John speec...
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,0.0484,0.000153,11,1,63.299,-10.537,"Put a candle in the window, 'cause I feel I'v...",...,-2,-9,-7,0,-7,2,0,-7,2,name: Long As I Can See The Light artists: Cre...
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,0.0597,0.002430,9,1,159.656,-6.926,"She moves like she don't care Smooth as silk, ...",...,-2,-4,-9,7,14,-16,7,-2,-5,name: Maria artists: Blondie speechiness: 0.05...


# Text for embeddings to chat gpt

In [72]:
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API en OpenAI
OpenAI.api_key = api_key

In [73]:
# Convert text into vectors using embeddings
client = OpenAI(
 api_key=api_key,
)

Input:

In [74]:
input_expanded['embedding'] = input_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [75]:
input_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [76]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me, I...",...,2,4,-3,7,-2,-5,5,2,-7,name: Mr. Tambourine Man artists: Bob Dylan sp...
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...",...,-2,5,0,-7,7,-5,-7,7,-5,name: I Saw Her Standing There - Remastered 20...
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young Me and Suzie ha...,...,0,5,5,2,-7,5,-7,5,0,name: Crocodile Rock artists: Elton John speec...
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,0.0484,0.000153,11,1,63.299,-10.537,"Put a candle in the window, 'cause I feel I'v...",...,-2,-9,-7,0,-7,2,0,-7,2,name: Long As I Can See The Light artists: Cre...
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,0.0597,0.002430,9,1,159.656,-6.926,"She moves like she don't care Smooth as silk, ...",...,-2,-4,-9,7,14,-16,7,-2,-5,name: Maria artists: Blondie speechiness: 0.05...


Related songs:

In [77]:
#para cada fila del dataframe, sacar el vector de embedding de la columna "to_embbed_text"
df_vectorial_expanded['embedding'] = df_vectorial_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [78]:
df_vectorial_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [79]:
df_vectorial_expanded = df_vectorial_expanded.rename(columns={'embedding': 'values'})

# Data to pinecone

In [80]:

# Procesar el DataFrame
df_vectorial_expanded = create_metadata_column(df_vectorial_expanded)
df_vectorial_expanded = clean_metadata_column(df_vectorial_expanded)
df_para_pinecone = create_dataframe_for_pinecone(df_vectorial_expanded)
insert_into_pinecone(df_para_pinecone, index)



sending upsert requests:   0%|          | 0/5 [00:00<?, ?it/s]

In [81]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,values,metadata
0,3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,Bob Dylan,0.0676,0.000000,5,1,177.515,-14.545,"Hey! Mr. Tambourine Man, play a song for me, I...",...,4,-3,7,-2,-5,5,2,-7,"[0.0128581328317523, 0.005557615775614977, -0....","{'name': 'Mr. Tambourine Man', 'artists': 'Bob..."
1,3KiexfmhxHvG5IgAElmTkd,I Saw Her Standing There - Remastered 2009,The Beatles,0.0361,0.000000,4,1,160.109,-9.835,"The theme of ""I Saw Her Standing There"" revolv...",...,5,0,-7,7,-5,-7,7,-5,"[0.017372271046042442, -0.020276140421628952, ...",{'name': 'I Saw Her Standing There - Remastere...
2,6WCeFNVAXUtNczb7lqLiZU,Crocodile Rock,Elton John,0.0344,0.000015,4,0,149.908,-7.284,I remember when rock was young Me and Suzie ha...,...,5,5,2,-7,5,-7,5,0,"[0.022265547886490822, 0.028396641835570335, 0...","{'name': 'Crocodile Rock', 'artists': 'Elton J..."
3,1Vu8p3k4rfbETQCjTGjmzg,Long As I Can See The Light,Creedence Clearwater Revival,0.0484,0.000153,11,1,63.299,-10.537,"Put a candle in the window, 'cause I feel I'v...",...,-9,-7,0,-7,2,0,-7,2,"[0.030892042443156242, 0.028187209740281105, 0...","{'name': 'Long As I Can See The Light', 'artis..."
4,7bOYOD0d7ByMTXvJnQVl8d,Maria,Blondie,0.0597,0.002430,9,1,159.656,-6.926,"She moves like she don't care Smooth as silk, ...",...,-4,-9,7,14,-16,7,-2,-5,"[0.010735495947301388, 0.027548208832740784, -...","{'name': 'Maria', 'artists': 'Blondie', 'speec..."


In [82]:
df_para_pinecone

,id,values,metadata
0,3RkQ3UwOyPqpIiIvGVewuU,"[0.0128581328317523, 0.005557615775614977, -0....","{'name': 'Mr. Tambourine Man', 'artists': 'Bob..."
1,3KiexfmhxHvG5IgAElmTkd,"[0.017372271046042442, -0.020276140421628952, ...",{'name': 'I Saw Her Standing There - Remastere...
2,6WCeFNVAXUtNczb7lqLiZU,"[0.022265547886490822, 0.028396641835570335, 0...","{'name': 'Crocodile Rock', 'artists': 'Elton J..."
3,1Vu8p3k4rfbETQCjTGjmzg,"[0.030892042443156242, 0.028187209740281105, 0...","{'name': 'Long As I Can See The Light', 'artis..."
4,7bOYOD0d7ByMTXvJnQVl8d,"[0.010735495947301388, 0.027548208832740784, -...","{'name': 'Maria', 'artists': 'Blondie', 'speec..."


# Query search in pinecone

In [83]:
response = realizar_consulta(client, input_expanded, index, top_k=5)

Resultado de la consulta: {'matches': [{'id': '1rqGuE3jvcvacQk96lfnl6',
              'metadata': {'artists': 'Geese',
                           'chords_1_1': 'G',
                           'chords_1_2': 'D',
                           'chords_1_3': 'Em',
                           'chords_1_4': 'C',
                           'chords_1_numeric_1': 7.0,
                           'chords_1_numeric_2': 2.0,
                           'chords_1_numeric_3': 16.0,
                           'chords_1_numeric_4': 0.0,
                           'chords_2_1': 'Am',
                           'chords_2_2': 'F',
                           'chords_2_3': 'C',
                           'chords_2_4': 'G',
                           'chords_2_numeric_1': 21.0,
                           'chords_2_numeric_2': 5.0,
                           'chords_2_numeric_3': 0.0,
                           'chords_2_numeric_4': 7.0,
                           'diff_chords_1_numeric_1': -5.0,
                 

In [84]:
response_matches = response['matches']

details = []
for match in response_matches:
    details.append(f"Intruments: {match['metadata']['instruments_1']} {match['metadata']['instruments_2']}, Tempo: {match['metadata']['tempo']}, Speechiness: {match['metadata']['speechiness']}, Loudness: {match['metadata']['loudness']}")

In [85]:
details

['Intruments: guitar bass, Tempo: 131.948, Speechiness: 0.0382, Loudness: -7.699',
 'Intruments: piano cello, Tempo: 133.355, Speechiness: 0.033, Loudness: -12.636',
 'Intruments: piano drums, Tempo: 124.357, Speechiness: 0.0678, Loudness: -7.621',
 'Intruments: piano guitar, Tempo: 114.032, Speechiness: 0.0388, Loudness: -6.153',
 'Intruments: guitar drums, Tempo: 135.168, Speechiness: 0.0643, Loudness: -2.719']

In [86]:

for match in response_matches:
    print(f"Letra: {match['metadata']['theme']}, \n Score: {match['score']}")

Letra: I’ve been hit by the bus of love That falling brick of you Baby, when there’s tears in your eyes You don’t have to lie, 
 Score: 0.19463329
Letra: Love and learn Take your turn Somewhere there's a feather Falling slowly from the sky, 
 Score: 0.188229024
Letra: Oh-oh-oh-oh Mhmm, mhmm From Bamenda, it's Libianca Oh-oh, 
 Score: 0.185013577
Letra: The theme of "Miles and Miles of Texas" revolves around the deep connection to the vast Texas landscape and the sense of freedom it brings to the narrator. It reflects a longing for home and the cherished memories tied to the places and experiences that define Texan identity., 
 Score: 0.179133505
Letra: Was that supposed to mean you're making no sense? Drinking all the time but now your just repeating yourself, Was that supposed to mean I think you might of made a mistak? Your eyes appear to be closed but I know that you're still awake., 
 Score: 0.179057181


In [87]:
diff_chord_2= input_data['diff_chords_2_numeric'].iloc[0]
diff_chord_1= input_data['diff_chords_1_numeric'].iloc[0]

In [88]:
# Crear el embedding de la nueva query
chords_query = f"A song with a similar progession and distance (diff) in chords to {input_data['chords_1'][0]},y diff chords {diff_chord_2},{diff_chord_1}."
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=chords_query
).data[0].embedding

chords_query = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

chords_query_ = chords_query['matches']



In [89]:
for match in chords_query_:
    print(f"Chords: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}, \n Score: {match['score']}")

Chords: C G Am F, F Am C G, 
 Score: 0.476434261
Chords: C G Am F, C F G Am, 
 Score: 0.466786236
Chords: C G Am F, D A Bm G, 
 Score: 0.459645778
Chords: C G Am F, C G F F, 
 Score: 0.459454
Chords: 2023 Remaster C G Am, F C G F, 
 Score: 0.453208357


In [90]:
chord_wheels=[]
for match in chords_query_:
    chord_wheels.append(f"Chord wheel: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}")

In [91]:
chord_wheels

['Chord wheel: C G Am F, F Am C G',
 'Chord wheel: C G Am F, C F G Am',
 'Chord wheel: C G Am F, D A Bm G',
 'Chord wheel: C G Am F, C G F F',
 'Chord wheel: 2023 Remaster C G Am, F C G F']

In [92]:
input_data['theme'][0]

'The song "Let It Be" by The Beatles conveys themes of comfort and acceptance in the face of uncertainty and challenges. It emphasizes the importance of letting go and finding peace amidst life\'s difficulties, often invoking the idea of guidance and reassurance in times of trouble. The refrain serves as a reminder that, even in moments of despair, there is a sense of hope and solace that can be found by simply allowing things to unfold naturally. Overall, the song promotes a message of resilience and the wisdom of surrender.'

In [93]:
# Crear el embedding de la nueva query
lyrics_query = f"Give me a similar theme song to {input_data['theme'][0]}"
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=lyrics_query
).data[0].embedding

lyrics_result = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

lyrics_matches = lyrics_result['matches']



In [94]:
lyrics_list = []  # Crear una lista vacía para almacenar los textos

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    lyrics_list.append(f"Lyrics: {match['metadata']['theme']}")



for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    print(f"Lyrics: {match['metadata']['theme']} {match['id']} Score: {match['score']}")

lyrics_list.append(input_data['theme'][0])

Lyrics: The theme of "Octopus's Garden" revolves around the idea of finding comfort and happiness in a peaceful and idyllic setting, away from the complexities of life. The lyrics describe a whimsical underwater paradise where one can escape from worries and enjoy simple pleasures, symbolized by the playful octopus. The song conveys a sense of innocence and childlike wonder, emphasizing the joy of togetherness and contentment in nature. Ultimately, it celebrates the freedom to dream and create a personal sanctuary amidst life's chaos. 0suLngfo7rJoetk7Ub6N8l Score: 0.525733292
Lyrics: "The Long and Winding Road" reflects themes of longing, introspection, and the emotional journey of life. The lyrics convey a sense of searching for solace and closure, illustrating the struggles and challenges faced along the way. The imagery of a winding road symbolizes the unpredictable nature of life and relationships, emphasizing the desire for connection and fulfillment despite obstacles. Ultimately,

In [95]:
lyrics_list

['Lyrics: The theme of "Octopus\'s Garden" revolves around the idea of finding comfort and happiness in a peaceful and idyllic setting, away from the complexities of life. The lyrics describe a whimsical underwater paradise where one can escape from worries and enjoy simple pleasures, symbolized by the playful octopus. The song conveys a sense of innocence and childlike wonder, emphasizing the joy of togetherness and contentment in nature. Ultimately, it celebrates the freedom to dream and create a personal sanctuary amidst life\'s chaos.',
 'Lyrics: "The Long and Winding Road" reflects themes of longing, introspection, and the emotional journey of life. The lyrics convey a sense of searching for solace and closure, illustrating the struggles and challenges faced along the way. The imagery of a winding road symbolizes the unpredictable nature of life and relationships, emphasizing the desire for connection and fulfillment despite obstacles. Ultimately, the song captures a poignant sens

In [96]:
nueva_cancion = create_song(lyrics_list, chord_wheels, details)
print(nueva_cancion)

# Título de la canción: El Jardín de los Sueños

**Tempo:** 124 BPM  
**Instrumentos principales:** piano, batería

### Letra y acordes:

1. **Verso 1**:  
   C             G  
   Bajo el cielo azul,  
   Am           F  
   donde los sueños crecen,  
   C               G  
   un rincón sin fin,  
   F                 Am  
   donde la paz florece.

2. **Coro**:  
   C                  G  
   En el jardín de los sueños,  
   Am                 F  
   donde el tiempo se va,  
   C                  G  
   juntos encontramos calma,  
   F                 C  
   sin mirar atrás.

3. **Verso 2**:  
   C                G  
   Caminamos despacio,  
   Am             F  
   por senderos de paz,  
   C                G  
   el viento nos canta,  
   F               Am  
   un himno de libertad.

4. **Coro final**:  
   C                  G  
   En el jardín de los sueños,  
   Am                  F  
   donde el tiempo no está,  
   C                  G  
   la alegría es nuestra guía,  
   F   

In [97]:
nueva_cancion

'# Título de la canción: El Jardín de los Sueños\n\n**Tempo:** 124 BPM  \n**Instrumentos principales:** piano, batería\n\n### Letra y acordes:\n\n1. **Verso 1**:  \n   C             G  \n   Bajo el cielo azul,  \n   Am           F  \n   donde los sueños crecen,  \n   C               G  \n   un rincón sin fin,  \n   F                 Am  \n   donde la paz florece.\n\n2. **Coro**:  \n   C                  G  \n   En el jardín de los sueños,  \n   Am                 F  \n   donde el tiempo se va,  \n   C                  G  \n   juntos encontramos calma,  \n   F                 C  \n   sin mirar atrás.\n\n3. **Verso 2**:  \n   C                G  \n   Caminamos despacio,  \n   Am             F  \n   por senderos de paz,  \n   C                G  \n   el viento nos canta,  \n   F               Am  \n   un himno de libertad.\n\n4. **Coro final**:  \n   C                  G  \n   En el jardín de los sueños,  \n   Am                  F  \n   donde el tiempo no está,  \n   C                  G